# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [33]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
import string
import numpy as np

In [2]:
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /Users/victorl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/victorl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victorl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('labeled_messages', engine)

In [357]:
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis=1)
category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [5]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
remove_punc_table = str.maketrans('', '', string.punctuation)

In [6]:
def tokenize(text):
    # normalize case and remove punctuation
    text = text.translate(remove_punc_table).lower()
    
    # tokenize text
    tokens = nltk.word_tokenize(text)
    
    # lemmatize and remove stop words
    return [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]



### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

In [436]:
forest_clf = RandomForestClassifier(n_estimators=10)

In [9]:
pipeline = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('forest', MultiOutputClassifier(forest_clf))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [14]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
from sklearn.metrics import classification_report

In [395]:
Y_pred = pipeline.predict(X_test)

In [17]:
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))

related
              precision    recall  f1-score   support

         0.0       0.34      0.14      0.20      2096
         1.0       0.77      0.91      0.83      6497

   micro avg       0.72      0.72      0.72      8593
   macro avg       0.55      0.53      0.52      8593
weighted avg       0.66      0.72      0.68      8593

request
              precision    recall  f1-score   support

         0.0       0.84      0.97      0.90      7117
         1.0       0.38      0.08      0.13      1476

   micro avg       0.82      0.82      0.82      8593
   macro avg       0.61      0.53      0.52      8593
weighted avg       0.76      0.82      0.77      8593

offer
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      8547
         1.0       0.00      0.00      0.00        46

   micro avg       0.99      0.99      0.99      8593
   macro avg       0.50      0.50      0.50      8593
weighted avg       0.99      0.99      0.99      859

/Users/victorl/code/udacity/DSND_Term2/disaster_pipeline/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      8231
         1.0       0.00      0.00      0.00       362

   micro avg       0.96      0.96      0.96      8593
   macro avg       0.48      0.50      0.49      8593
weighted avg       0.92      0.96      0.94      8593

other_aid
              precision    recall  f1-score   support

         0.0       0.87      0.98      0.92      7497
         1.0       0.13      0.02      0.03      1096

   micro avg       0.86      0.86      0.86      8593
   macro avg       0.50      0.50      0.48      8593
weighted avg       0.78      0.86      0.81      8593

infrastructure_related
              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97      8066
         1.0       0.05      0.00      0.01       527

   micro avg       0.93      0.93      0.93      8593
   macro avg       0.50      0.50      0.49      8593
weighted avg       0.88      0.93      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
from sklearn.model_selection import GridSearchCV

In [458]:
parameters = {
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'tfidf__max_df': (0.8, 1.0),
    'tfidf__max_features': (None, 10000),
    'forest__estimator__n_estimators': [50, 100],
    'forest__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1)

In [459]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [364]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [451]:
def evaluate_model(model, X_test, Y_test):
    Y_pred = model.predict(X_test)
    print(classification_report(Y_test, Y_pred, target_names=category_names))
#     print('Accuracy: ', accuracy_score(Y_test, Y_pred))
#     print('Precision: ', precision_score(Y_test, Y_pred, average='weighted'))
#     print('Recall: ', recall_score(Y_test, Y_pred, average='weighted'))

In [129]:
print('Accuracy: ', accuracy_score(Y_test, Y_pred))
print('Precision: ', precision_score(Y_test, Y_pred, average='weighted'))
print('Recall: ', recall_score(Y_test, Y_pred, average='weighted'))

Accuracy:  0.14802746421505877
Precision:  0.3900151724208853
Recall:  0.27685873949890827


/Users/victorl/code/udacity/DSND_Term2/disaster_pipeline/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [352]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD

In [206]:
def is_null(series):

    def is_null_base(s):
        res = 0 if s is None else 1
        return res
    
    final = pd.DataFrame(series.applymap(is_null_base))
    return final

In [354]:
categorical_features = ['genre']
categorical_transform = OneHotEncoder()

original = ['original']
original_transform = FunctionTransformer(is_null, validate=False)

text = 'message'
text_transform = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                           ('best', TruncatedSVD(n_components=50))])

In [355]:
preprocessor = ColumnTransformer(transformers=[
                                ('cat', categorical_transform, categorical_features),
                                ('orig', original_transform, original),
                                ('tfidf', text_transform, 'message')
                                              ])

In [356]:
preprocessor.fit_transform(X_2)

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.92220547e-02, -4.33138260e-02, -2.60150053e-02],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         3.10686205e-02, -1.99762261e-02,  1.34438408e-02],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33582327e-02, -3.16578759e-02,  4.13029577e-02],
       ...,
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         3.37286416e-02,  8.27223301e-03, -5.65770821e-03],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         6.45141232e-04, -7.53957391e-03,  8.45396210e-03],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
        -5.62647930e-03,  2.18488046e-02, -3.19588149e-02]])

After working on this for a bit I decided to check how the code for the webserver would work and noted that there was a section that took new user input - this won't have `original` or `genre` so this entire column transformer is pretty moot. From here I'm going to go back to just using the tfidf + some other text features.

In [461]:
clf = forest_clf

In [462]:
pipeline_2 = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('best', TruncatedSVD(n_components=100)),
                    ('clf', MultiOutputClassifier(clf))
                      ])

In [455]:
# test to make sure this works ok
pipeline_2.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

In [468]:
print(pipeline_2.steps)

[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x1027411e0>, use_idf=True,
        vocabulary=None)), ('best', TruncatedSVD(algorithm='randomized', n_components=100, n_iter=5,
       random_state=None, tol=0.0)), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_

In [456]:
evaluate_model(pipeline_2, X_test, Y_test)

                        precision    recall  f1-score   support

               related       0.77      0.87      0.82      6497
               request       0.38      0.12      0.18      1476
                 offer       0.00      0.00      0.00        46
           aid_related       0.43      0.25      0.31      3488
          medical_help       0.19      0.01      0.01       669
      medical_products       0.11      0.00      0.00       432
     search_and_rescue       0.25      0.00      0.01       245
              security       0.00      0.00      0.00       167
              military       0.00      0.00      0.00       265
           child_alone       0.00      0.00      0.00         0
                 water       0.07      0.00      0.00       555
                  food       0.19      0.02      0.03       970
               shelter       0.10      0.01      0.01       760
              clothing       0.00      0.00      0.00       140
                 money       0.00      

/Users/victorl/code/udacity/DSND_Term2/disaster_pipeline/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/victorl/code/udacity/DSND_Term2/disaster_pipeline/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/victorl/code/udacity/DSND_Term2/disaster_pipeline/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/victorl/code/udacity/DSND_Term2/disaster_pipeline/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1

In [482]:
param_grid = {
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'tfidf__max_df': (0.8, 1.0),
    'tfidf__max_features': (None, 10000),
    'best__n_components': (10, 50, 100),
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 4]
}

In [485]:
cv_2 = GridSearchCV(pipeline_2, param_grid, cv=3, verbose=10, n_jobs=-1)

In [486]:
cv_2.fit(X_train, Y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 15.7min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 21.3min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 25.2min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 31.9min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 38.7min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 42.7min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 60.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 71.5min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 80.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 95

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'tfidf__ngram_range': ((1, 1), (1, 2)), 'tfidf__max_df': (0.8, 1.0), 'tfidf__max_features': (None, 10000), 'best__n_components': (10, 50, 100), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [494]:
from pprint import pprint

In [495]:
pprint(cv_2.best_estimator_.steps)

[('tfidf',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x1027411e0>, use_idf=True,
        vocabulary=None)),
 ('best',
  TruncatedSVD(algorithm='randomized', n_components=100, n_iter=5,
       random_state=None, tol=0.0)),
 ('clf',
  MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
      

In [491]:
evaluate_model(cv_2.best_estimator_, X_test, Y_test)

                        precision    recall  f1-score   support

               related       0.76      0.98      0.86      6497
               request       0.35      0.05      0.08      1476
                 offer       0.00      0.00      0.00        46
           aid_related       0.45      0.18      0.26      3488
          medical_help       0.00      0.00      0.00       669
      medical_products       0.00      0.00      0.00       432
     search_and_rescue       0.00      0.00      0.00       245
              security       0.00      0.00      0.00       167
              military       0.00      0.00      0.00       265
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       555
                  food       0.11      0.00      0.00       970
               shelter       0.21      0.00      0.01       760
              clothing       0.00      0.00      0.00       140
                 money       0.00      

/Users/victorl/code/udacity/DSND_Term2/disaster_pipeline/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/victorl/code/udacity/DSND_Term2/disaster_pipeline/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/victorl/code/udacity/DSND_Term2/disaster_pipeline/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/victorl/code/udacity/DSND_Term2/disaster_pipeline/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1

### 9. Export your model as a pickle file

In [368]:
import joblib

In [490]:
joblib.dump(cv_2.best_estimator_, 'disaster_model.pkl')

['disaster_model.pkl']

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.